# 2022年 大数据管理与应用20级 机器学习课程机考试题

## 学号：202031112050002
## 姓名：黄凯纯

#填写完成后运行该代码块，双击后可以继续修改

###### 说明
1、用合适的代码替换以下的*

2、请在代码中添加适当的注释   （共6分）

## 时间序列预测

数据集为某车型每月的销量数据，根据以下数据集，构建汽车销量模型，预测19年9月的汽车销量。

#以下导入常用的数据包，如果需要可自行导入。

In [1]:
import pandas  as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error 
%matplotlib inline

#### 读入数据（5分）

In [2]:
#数据集编码中文为utf-8，第一列的名字为“Unnamed: 0”
data=pd.read_csv("./汽车销量考试用.csv",  encoding="utf-8",  index_col="Unnamed: 0")

In [3]:
data.info()#查看数据集的信息

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 27 to 102
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年月      76 non-null     object 
 1   销量      74 non-null     float64
 2   月排名     76 non-null     int64  
 3   份额      76 non-null     object 
 4   厂商排名    76 non-null     int64  
 5   车型排名    76 non-null     int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.2+ KB


In [4]:
data.head()# 查看前5行数据集

,年月,销量,月排名,份额,厂商排名,车型排名
27,19-Aug,253.0,369,0.20%,16,79
28,19-Jul,480.0,319,0.39%,16,70
29,19-Jun,513.0,340,0.39%,16,74
30,19-May,235.0,373,0.18%,16,90
31,19-Apr,410.0,337,0.33%,15,77


#### 随机查看5个样本数据（3分）

In [5]:
data.sample(5)#随机查看5个样本

,年月,销量,月排名,份额,厂商排名,车型排名
57,17-Feb,170.0,330,0.14%,9,80
46,18-Jan,4837.0,145,2.70%,9,38
48,17-Nov,4592.0,158,2.73%,9,41
75,15-Aug,5070.0,92,4.57%,7,30
85,14-Oct,10153.0,56,8.77%,6,21


### 数据预处理

#### 缺失值处理（5分）

In [6]:
data=data.fillna( method="ffill")  #补充完整,采用ffill填补

#### 非数值数据处理(5分)

In [8]:
#补充完以下代码将百分比转化为对应的数值 1%=>0.01
data.份额=data.份额.map(lambda x : (x.split('%')[0])      )
  

In [9]:
data.head()#查看数据的前5行数据

,年月,销量,月排名,份额,厂商排名,车型排名
27,19-Aug,253.0,369,0.20,16,79
28,19-Jul,480.0,319,0.39,16,70
29,19-Jun,513.0,340,0.39,16,74
30,19-May,235.0,373,0.18,16,90
31,19-Apr,410.0,337,0.33,15,77


#### 非数值数据处理2 (2分)

#删除特征年月
del  

In [10]:
del data["年月"]  # 删除数据集中的年月

In [11]:
data.info()#查看删除年月后的数据集信息

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 27 to 102
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   销量      76 non-null     float64
 1   月排名     76 non-null     int64  
 2   份额      76 non-null     object 
 3   厂商排名    76 non-null     int64  
 4   车型排名    76 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 3.6+ KB


#### 时间序列数据格式转化换（5分）

In [12]:
#用前12月的数据预测下一个月
sequence_length=12*5
data_ = []
for i in range(  sequence_length+1  ):
    data_.append(data.销量[i: i + sequence_length +1])

In [13]:
data_ = np.array([df.values for df in data_])

C:\Users\morang\AppData\Local\Temp\ipykernel_3528\775303613.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_ = np.array([df.values for df in data_])


#### 预测的数据，取最后的12行，修改shape由（12，5）转为（1，12，5）以符合输入要求（3分）

In [15]:
X_pred=data[64:]#取data数据集的后12行数据

In [27]:
X_pred.shape# 查看数据的维数

(12, 5)

In [28]:
data_

array([array([  253.,   480.,   513.,   235.,   410.,   723.,   479.,   970.,
                164.,  1751.,  2857.,  3853.,  3647.,  2576.,  2918.,  3303.,
               4107.,  3859.,  1161.,  4837.,  5254.,  4592.,  8431.,  6177.,
               3758.,  1383.,  2552.,   213.,  7213.,  6560.,   170.,  7805.,
               8440.,  1885.,  5506.,  6984.,  4615.,  3619.,  5716.,  6759.,
               6815.,  5316.,  6341.,  7278.,  4775., 10234.,  8589.,  8751.,
               5070.,  5827.,  4899.,  8865.,  6338.,  4773.,  9364., 16278.,
               8713.,  9272., 10153., 10450., 10450.])                       ,
       array([  480.,   513.,   235.,   410.,   723.,   479.,   970.,   164.,
               1751.,  2857.,  3853.,  3647.,  2576.,  2918.,  3303.,  4107.,
               3859.,  1161.,  4837.,  5254.,  4592.,  8431.,  6177.,  3758.,
               1383.,  2552.,   213.,  7213.,  6560.,   170.,  7805.,  8440.,
               1885.,  5506.,  6984.,  4615.,  3619.,  5716.,  

#### 乱序数据(5分)

In [29]:
np.random.shuffle(data_)#用np下的random.shuffle将数据集打乱

In [30]:
data_.shape#查看data_的数据形状

(61,)

#### X,y分离 （5分）

In [46]:
data_[1:2:1]

array([array([ 3303.,  4107.,  3859.,  1161.,  4837.,  5254.,  4592.,  8431.,
               6177.,  3758.,  1383.,  2552.,   213.,  7213.,  6560.,   170.,
               7805.,  8440.,  1885.,  5506.,  6984.,  4615.,  3619.,  5716.,
               6759.,  6815.,  5316.,  6341.,  7278.,  4775., 10234.,  8589.,
               8751.,  5070.,  5827.,  4899.,  8865.,  6338.,  4773.,  9364.,
              16278.,  8713.,  9272., 10153., 10450., 10450., 10443.,  3658.,
               8397., 10899., 10899., 10695., 11010.,  5421.,  6404.,  8791.,
              12315., 10247.,  8493.,  6122.,  2021.])                       ],
      dtype=object)

In [51]:
X=data_[::]

In [50]:
y=data_[::12]

#### 数据集划分（训练集（70%），验证集（15%），测试集（15%））（6分）

In [52]:
X_train= X[:int(len(X)*.7)]#X_train是X数据集的前70%的数据，加上int是防止数据集的划分不是整数导致报错
y_train= y[:int(len(y)*.7)]#y_train是y数据集的前70%的数据

In [53]:
X_val= X[int(len(X)*.7):int(len(X)*.85)]#X_val是X数据集的70%--85%的数据
y_val= y[int(len(y)*.7):int(len(y)*.85)]#y_val是y数据集的70%--85%的数据

In [54]:
X_test= X[int(len(X)*.85):]#X_test是X数据集的后85%的数据
y_test= y[int(len(y)*.85):]#y_test是y数据集的后85%的数据

### 1、全连接神经网络预测 或者 卷积神经网络预测  （共25分）

#### 构建模型  （8分）

In [56]:
from tensorflow import keras

In [59]:
model1=tf.keras.Sequential()#创建一个顺序模型
model1.add(keras.layers.Conv2D(X_train,y_train))
model1.add(keras.layers.BatchNormalization())
model1.add(keras.layers.Dense() )

TypeError: __init__() missing 1 required positional argument: 'kernel_size'

In [ ]:
model1.summary()#查看模型的层级

#### 编译模型 （5分）

In [ ]:
#参数自选
model1.compile(optimizer='adam' ,loss=tf.losses.sparse_categorical_crossentropy,metrics=['acc'])#设置一个优化器

#### 训练模型   （3分）

In [ ]:
#100轮，批次大小8，
his=model1.fit(X_train,y_train,epochs=100,validation_data=8)#进行数据的训练

## 可视化训练过程（4分）

In [ ]:
plt.plot(his.history.epoch,his.history.get('loss'),label='loss')#画出loss的图形
plt.plot(his.history.epoch,his.history.get('acc'),label='acc')#画出acc的图形
plt.legend()#显示图例

## 模型评价（2分）

In [ ]:
model1.evaluate(X_train,y_test)#根据训练集和测试集对模型进行评价

### 预测 （3分）

In [ ]:
#预测19年9月的汽车销量
model1.predict('19-Sept')

### 2、循环神经网络预测   （共25分）

###### 构建模型  （8分）

In [64]:
model2 = tf.keras.Sequential()
model2.add(keras.layers.RNN(X_train,y_train,activity='relu')
model2.add(keras.layers.Dense((64),activity='adam')
model2.add(keras.layers.Dense((4),activity='relu')
#按需添加layers

SyntaxError: invalid syntax (940582795.py, line 3)

#### 编译模型 （5分）

In [ ]:
model2.compile(optimizer='adam' ,loss=tf.losses.serialize ,metrics=['acc'])#设置一个优化器

#### 训练模型   （3分）

In [ ]:
his2=model2.fit(X_train,y_train,epochs=20,validation_data=8)#训练模型

## 可视化训练过程（4分）

In [ ]:
plt.plot(his2.history.epoch,his2.history.get('loss'),label='loss')#画出loss的图形
plt.plot(his2.history.epoch,his2.history.get('acc'),label='acc')#画出acc的图形
plt.legend()

## 模型评价（2分）

In [ ]:
model2.evaluate(X_train,y_test)#根据训练集和测试集对模型进行评价

## 预测 （3分）

In [ ]:
model2.predict(X_test,y_test)#对模型做一个预测